# Stock Price and Earnings Event Analysis
This notebook prepares daily stock price data and earnings announcement dates for Apple, Google, and Nvidia (2005–2025) and calculates daily returns in preparation for further analysis.

The dataset is stored as event_study_price_data.csv.

## 1. Load data
Two data sets are loaded: 
- daily stock price data (open, high, low, close) for Apple, Google, and Nvidia
- earnings announcement dates 

In [46]:
import pandas as pd

In [47]:
prices = pd.read_csv("ohlc_v2.csv")
earnings = pd.read_csv("earnings_dates_v3.csv")

In [48]:
prices.head()

,ticker,date,open,high,low,close,volume
0,AAPL,2005/01/03,1.16,1.16,1.12,1.13,12382431
1,AAPL,2005/01/04,1.14,1.17,1.12,1.14,19608238
2,AAPL,2005/01/05,1.15,1.17,1.14,1.15,12157876
3,AAPL,2005/01/06,1.16,1.16,1.13,1.15,12604964
4,AAPL,2005/01/07,1.16,1.24,1.16,1.24,39923768


In [49]:
earnings.head(100)

,ticker,earnings_date
0,AAPL,2005/01/12
1,AAPL,2005/04/13
2,AAPL,2005/07/13
3,AAPL,2005/10/11
4,AAPL,2006/01/18
...,...,...
95,GOOGL,2007/10/18
96,GOOGL,2008/01/31
97,GOOGL,2008/04/17
98,GOOGL,2008/07/17


## 2. Data type preparation
Date columns are converted to datetime  format to enable time-based calculations. 

In [50]:
# Store date columns as datetime objects.
prices['date'] = pd.to_datetime(prices['date'])

In [51]:
# Check data types
prices.dtypes

ticker            object
date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
dtype: object

In [52]:
earnings['earnings_date'] = pd.to_datetime(earnings['earnings_date'])

In [53]:
earnings.dtypes

ticker                   object
earnings_date    datetime64[ns]
dtype: object

## 3. Merge earnings dates with stock price data
In this section, the dataset is prepared for event-study analysis by merging earnings announcement dates with daily stock price data. We first ensure that price data are sorted chronologically within each firm. Earnings dates are then merged into the price dataset so that the stock price on each earnings announcement date can be identified.

In [54]:
# Sort data appropriately.
prices = prices.sort_values(['ticker', 'date'])
earnings = earnings.sort_values(['ticker', 'earnings_date'])

In [55]:
prices[['ticker','date']].head(5)

,ticker,date
0,AAPL,2005-01-03
1,AAPL,2005-01-04
2,AAPL,2005-01-05
3,AAPL,2005-01-06
4,AAPL,2005-01-07


In [56]:
earnings.head(10)

,ticker,earnings_date
0,AAPL,2005-01-12
1,AAPL,2005-04-13
2,AAPL,2005-07-13
3,AAPL,2005-10-11
4,AAPL,2006-01-18
5,AAPL,2006-04-19
6,AAPL,2006-07-19
7,AAPL,2006-10-18
8,AAPL,2007-01-17
9,AAPL,2007-04-25


In [57]:
# Merge earnings dates into prices.
prices = prices.merge(earnings, left_on=['ticker', 'date'], right_on=['ticker', 'earnings_date'], how='left')

In [58]:
# Check earnings dates have merged.
prices[['ticker','date','earnings_date']].dropna().head()

,ticker,date,earnings_date
7,AAPL,2005-01-12,2005-01-12
69,AAPL,2005-04-13,2005-04-13
132,AAPL,2005-07-13,2005-07-13
195,AAPL,2005-10-11,2005-10-11
262,AAPL,2006-01-18,2006-01-18


In [59]:
# After sorting by ticker and date, we assign each row a sequential trading-day number (td_num) within each ticker, 
# in order to identify days before and after an earnings event based on trading-day position rather than calendar dates.
prices = prices.sort_values(['ticker','date']).reset_index(drop=True)
prices['td_num'] = prices.groupby('ticker').cumcount()

In [60]:
prices[['ticker','date','td_num']].head(5)

,ticker,date,td_num
0,AAPL,2005-01-03,0
1,AAPL,2005-01-04,1
2,AAPL,2005-01-05,2
3,AAPL,2005-01-06,3
4,AAPL,2005-01-07,4


## 4. Construct Earnings Event Windows (Trading Days)
We create an event-day index for each earnings announcement. Day 0 represents the earnings announcement date. Trading days before and after the event are labelled in relation to this date (-5 to +5), enabling analysis of return behaviour around earnings events.

In [61]:
# Create an "events" table containing only earnings days and their trading-day position
events = prices.loc[
    prices['earnings_date'].notna(),
    ['ticker', 'earnings_date', 'td_num']
].rename(columns={'td_num': 'event_td_num'})

events.head(5)

,ticker,earnings_date,event_td_num
7,AAPL,2005-01-12,7
69,AAPL,2005-04-13,69
132,AAPL,2005-07-13,132
195,AAPL,2005-10-11,195
262,AAPL,2006-01-18,262


In [62]:
# Define the event window offsets (-5 to +5 trading days)
window = pd.DataFrame({'event_day': range(-5, 6)})
window

,event_day
0,-5
1,-4
2,-3
3,-2
4,-1
5,0
6,1
7,2
8,3
9,4


In [63]:
# Expand each earnings event into -5...+5 event-day rows
events_expanded = events.merge(window, how='cross')

# Compute the trading-day number for each event-day offset
events_expanded['td_num'] = events_expanded['event_td_num'] + events_expanded['event_day']

events_expanded.head(11)

,ticker,earnings_date,event_td_num,event_day,td_num
0,AAPL,2005-01-12,7,-5,2
1,AAPL,2005-01-12,7,-4,3
2,AAPL,2005-01-12,7,-3,4
3,AAPL,2005-01-12,7,-2,5
4,AAPL,2005-01-12,7,-1,6
5,AAPL,2005-01-12,7,0,7
6,AAPL,2005-01-12,7,1,8
7,AAPL,2005-01-12,7,2,9
8,AAPL,2005-01-12,7,3,10
9,AAPL,2005-01-12,7,4,11


In [64]:
# Convert event windows into real calendar dates
event_day_labels = events_expanded.merge(
    prices[['ticker', 'td_num', 'date']],  
    on=['ticker', 'td_num'],
    how='left'
)[['ticker', 'date', 'event_day']]

event_day_labels.head(11)

,ticker,date,event_day
0,AAPL,2005-01-05,-5
1,AAPL,2005-01-06,-4
2,AAPL,2005-01-07,-3
3,AAPL,2005-01-10,-2
4,AAPL,2005-01-11,-1
5,AAPL,2005-01-12,0
6,AAPL,2005-01-13,1
7,AAPL,2005-01-14,2
8,AAPL,2005-01-18,3
9,AAPL,2005-01-19,4


In [65]:
# Attach event-day labels to the full trading history
prices = prices.merge(
    event_day_labels,
    on=['ticker', 'date'],
    how='left'
)

prices.head(11)

,ticker,date,open,high,low,close,volume,earnings_date,td_num,event_day
0,AAPL,2005-01-03,1.16,1.16,1.12,1.13,12382431,NaT,0,NaN
1,AAPL,2005-01-04,1.14,1.17,1.12,1.14,19608238,NaT,1,NaN
2,AAPL,2005-01-05,1.15,1.17,1.14,1.15,12157876,NaT,2,-5.0
3,AAPL,2005-01-06,1.16,1.16,1.13,1.15,12604964,NaT,3,-4.0
4,AAPL,2005-01-07,1.16,1.24,1.16,1.24,39923768,NaT,4,-3.0
5,AAPL,2005-01-10,1.25,1.26,1.21,1.23,31018490,NaT,5,-2.0
6,AAPL,2005-01-11,1.22,1.23,1.15,1.15,46716924,NaT,6,-1.0
7,AAPL,2005-01-12,1.17,1.18,1.13,1.17,36249704,2005-01-12,7,0.0
8,AAPL,2005-01-13,1.32,1.33,1.25,1.25,56600952,NaT,8,1.0
9,AAPL,2005-01-14,1.25,1.28,1.24,1.25,31631950,NaT,9,2.0


## 5. Calculate daily returns

In [66]:
prices = prices.sort_values(['ticker', 'date'])

prices['daily_return'] = (
    prices.groupby('ticker')['close']
    .pct_change()
)

prices['daily_return_pct'] = prices['daily_return'] * 100

prices.head(11)

,ticker,date,open,high,low,close,volume,earnings_date,td_num,event_day,daily_return,daily_return_pct
0,AAPL,2005-01-03,1.16,1.16,1.12,1.13,12382431,NaT,0,NaN,NaN,NaN
1,AAPL,2005-01-04,1.14,1.17,1.12,1.14,19608238,NaT,1,NaN,0.008850,0.884956
2,AAPL,2005-01-05,1.15,1.17,1.14,1.15,12157876,NaT,2,-5.0,0.008772,0.877193
3,AAPL,2005-01-06,1.16,1.16,1.13,1.15,12604964,NaT,3,-4.0,0.000000,0.000000
4,AAPL,2005-01-07,1.16,1.24,1.16,1.24,39923768,NaT,4,-3.0,0.078261,7.826087
5,AAPL,2005-01-10,1.25,1.26,1.21,1.23,31018490,NaT,5,-2.0,-0.008065,-0.806452
6,AAPL,2005-01-11,1.22,1.23,1.15,1.15,46716924,NaT,6,-1.0,-0.065041,-6.504065
7,AAPL,2005-01-12,1.17,1.18,1.13,1.17,36249704,2005-01-12,7,0.0,0.017391,1.739130
8,AAPL,2005-01-13,1.32,1.33,1.25,1.25,56600952,NaT,8,1.0,0.068376,6.837607
9,AAPL,2005-01-14,1.25,1.28,1.24,1.25,31631950,NaT,9,2.0,0.000000,0.000000


## 6. Save as CSV file

In [67]:
# Save the DataFrame as a CSV file
prices.to_csv("event_study_price_data_v2.csv", index=False)